In [1]:
%matplotlib notebook
%matplotlib inline

import axs
import numpy as np

from astropy.io import fits
import astropy.coordinates as coord
import astropy.units as u

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def spark_start(local_dir):
    from pyspark.sql import SparkSession
    
    spark = (
            SparkSession.builder
            .appName("LSD2")
            .config("spark.sql.warehouse.dir", local_dir)
            .config('spark.master', "local[6]")
            .config('spark.driver.memory', '8G') # 128
            .config('spark.local.dir', local_dir)
            .config('spark.memory.offHeap.enabled', 'true')
            .config('spark.memory.offHeap.size', '4G') # 256
            .config("spark.sql.execution.arrow.enabled", "true")
            .config("spark.driver.maxResultSize", "6G")
            .config("spark.driver.extraJavaOptions", f"-Dderby.system.home={local_dir}")
            .enableHiveSupport()
            .getOrCreate()
                    )   

    return spark

spark_session = spark_start("/epyc/users/ecbellm/spark-tmp/")

catalog = axs.AxsCatalog(spark_session)


In [3]:
spark_session

Hovering over "Spark UI" above gives you the port number of the Spark web dashboard.  Epyc doesn't have that port open, though, so we use an SSH tunnel to forward the ports.  I like to put the following function into my `.bashrc` o my local machine:


```
function spark_tunnel()
{
        # this function takes one argument: the epyc port to tunnel
        # the ordering is backwards (requiring a manual refresh) because
        # I want to be able to manually kill the ssh tunnel
        open http://localhost:${1}/
        ssh -N -L ${1}:127.0.0.1:${1} username@epyc.astro.washington.edu
}
```

What tables does AXS know about?

In [4]:
catalog.list_tables().keys()

dict_keys(['sdss_zoned1am_hd', 'gaia_zoned1am_hd', 'gaia_dr2_1am_dup', 'sdss_zoned1am_700lc', 'gaia_dr2_700lc2', 'allwise_1am_dup', 'gaia_sdss_wise_1asec', 'gaia_sdss_3asec', 'ztf_1am_old', 'ztf_exposures', 'ztf_1am_lc', 'ztf_1am', 'ztf_1am_test', 'sesar_rrlyrae', 'ztf_fid1_sdss_stars_lt20_2asec', 'ztf_fid2_sdss_stars_lt20_2asec', 'ztf_fid3_sdss_stars_lt20_2asec', 'sdss_dr9_qso_s82', 'faraway_training_dataset', 'faraway_labeled_training_dataset', 'ztf_nobs100', 'jnk_ztf_test', 'jnk_ztf_test2', 'ztf1000', 'ztf10', 'ztf_dec18', 'asassn_variability_catalog', 'ztf10_assasn', 'ztf10_assasn_cesium', 'ztf10_assasn_best', 'ztf10_assasn_cesium_best', 'unwise_v1', 'cklein_flare', 'unwise_v2', 'cesium_speedtest_ztfsample', 'paula_listcvs', 'nemec_rrlyrae_metalicity', 'dambis_rrlyrae_metalicity', 'sdss_500b_28e_10800z', 'gaia_500b_28e_10800z', 'allwise_500b_28e_10800z', 'ztf_500b_28e_10800z', 'ztf_mar19_all', 'ztf_dr1_s82_qso', 'green19_stellar_params', 'catalina_variables_n', 'caltech_rosat_perio

##  Exploring ZTF

In [5]:
ztf = catalog.load('ztf_mar19_all')

In [6]:
ztf.columns

['matchid',
 'ra',
 'dec',
 'nobs_avail',
 'combined_matchids',
 'mjd',
 'programid',
 'filterid',
 'mag',
 'magerr',
 'psfmag',
 'psfmagerr',
 'psfflux',
 'psffluxerr',
 'chi',
 'catflags',
 'sharp',
 'xpos',
 'ypos',
 'zone',
 'dup']

Look at the top 1 row:

In [23]:
ztf.head(1)

[Row(matchid=10294422001559, ra=0.029200815384615383, dec=-16.263277997435896, nobs_avail=39, combined_matchids=[10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294421013987, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559, 10294422001559], mjd=[58429.2307176, 58378.3356597, 58426.3169329, 58323.4388194, 58366.3977315, 58338.4080093, 58351.418831, 58345.4153588, 58423.2403356, 58342.436956, 58369.3298611, 58372.4009954, 58432.2429861, 58363.3348495, 58390.3144097, 58360.3542824, 58472.1060532, 5842

### Region query

In [24]:
%%time 
testra = 287.9869104
testdec = 13.0748496
d = 10/3600.

radec = ztf.region(ra1=testra-d, ra2=testra+d, dec1=testdec-d, dec2=testdec+d).select("ra", "dec").toPandas()


CPU times: user 10.7 ms, sys: 4.52 ms, total: 15.2 ms
Wall time: 19.4 s


In [25]:
radec

,ra,dec
0,287.985648,13.076047
1,287.985927,13.075574
2,287.986183,13.076572
3,287.986859,13.074891
4,287.988868,13.074120


`toPandas()` is convenient but requires bringing things into memory.  Can also do `collect()` and get a `list` of Spark `Rows`:

In [27]:
ztf.region(ra1=testra-d, ra2=testra+d, dec1=testdec-d, dec2=testdec+d).select("ra", "dec").collect()


[Row(ra=287.9856482260273, dec=13.076046963013694),
 Row(ra=287.98592663333335, dec=13.075574066666668),
 Row(ra=287.9861826000002, dec=13.076572423893802),
 Row(ra=287.9868585223528, dec=13.074891416862743),
 Row(ra=287.9888677572583, dec=13.074119947983874)]

AXS has a `cone` search but it is much slower [at present](https://github.com/dirac-institute/AXS/issues/4).

### Crossmatch to Gaia

In [18]:
gaia = catalog.load('gaia_dr2_1am_dup')

In [19]:
gaia.columns

['solution_id',
 'designation',
 'source_id',
 'random_index',
 'ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'parallax',
 'parallax_error',
 'parallax_over_error',
 'pmra',
 'pmra_error',
 'pmdec',
 'pmdec_error',
 'ra_dec_corr',
 'ra_parallax_corr',
 'ra_pmra_corr',
 'ra_pmdec_corr',
 'dec_parallax_corr',
 'dec_pmra_corr',
 'dec_pmdec_corr',
 'parallax_pmra_corr',
 'parallax_pmdec_corr',
 'pmra_pmdec_corr',
 'astrometric_n_obs_al',
 'astrometric_n_obs_ac',
 'astrometric_n_good_obs_al',
 'astrometric_n_bad_obs_al',
 'astrometric_gof_al',
 'astrometric_chi2_al',
 'astrometric_excess_noise',
 'astrometric_excess_noise_sig',
 'astrometric_params_solved',
 'astrometric_primary_flag',
 'astrometric_weight_al',
 'astrometric_pseudo_colour',
 'astrometric_pseudo_colour_error',
 'mean_varpi_factor_al',
 'astrometric_matched_observations',
 'visibility_periods_used',
 'astrometric_sigma5d_max',
 'frame_rotator_object_type',
 'matched_observations',
 'duplicated_source',
 'phot_g_n_ob

In [31]:
%%time
gaia_match = ztf.region(ra1=testra-d, ra2=testra+d, dec1=testdec-d, dec2=testdec+d).crossmatch(gaia).select(
    "matchid","ra","dec","source_id","bp_rp", "phot_g_mean_mag", "axsdist").toPandas()

CPU times: user 18.9 ms, sys: 6.01 ms, total: 24.9 ms
Wall time: 2.72 s


In [33]:
gaia_match

,matchid,ra,dec,source_id,bp_rp,phot_g_mean_mag,axsdist
0,10539322169560,287.985648,13.076047,4314064752141722880,1.962526,20.610218,2.010376e-06
1,10539322144433,287.985927,13.075574,4314064752144511104,NaN,21.399178,2.962654e-06
2,10539322095877,287.986183,13.076572,4314064747823789440,1.894012,20.256207,4.053908e-07
3,10539322095931,287.986859,13.074891,4314064752187515520,2.932623,19.596601,2.977570e-07
4,10539322022594,287.988868,13.074120,4314064752141447168,2.656782,20.182062,8.946068e-07
5,10539322022594,287.988868,13.074120,4314064747823772032,NaN,20.916426,4.717453e-06



### Filter results by value

TODO: show an example of filtering returned results

### Write lightcurves to disk

For up to 100s of thousands of lightcurves it's often most convenient to simply write out the matched results to disk.  Parquet is the best choice because the individual worker nodes can write independently.

In [34]:
lcmatch = ztf.region(ra1=testra-d, ra2=testra+d, dec1=testdec-d, dec2=testdec+d).select(
    'matchid', 'ra', 'dec', 'mjd', 'mag','magerr', 'filterid', 'catflags')

In [ ]:
lcmatch.write.parquet(f'./some_lightcurves.parquet')

### Example of creating a new AXS table

We need to get our table into a spark Dataframe.  There are several ways to get here, including reading from parquet:

In [ ]:
sdf = spark_session.read.parquet('some_objects.parquet')

Reading from a directory of csv files:

In [ ]:
%%time
# read a directory of files
sdf = spark_session.read.option("header", "true").option("inferSchema", "true").csv('/epyc/data/gaia_dr2_distances_csv/*.csv.gz')

Probably several more examples.  Once we have a spark dataframe, save as a new table:

In [ ]:
catalog.save_axs_table( sdf, 'new_table_name', repartition=True, calculate_zone=True)

If you need to drop a table:

In [ ]:
catalog.drop_table('new_table_name')


Note that you'll also probably have to manually delete the according directory at `/epyc/projects/lsd2/pzwarehouse/` 